In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import util
import numpy as np

In [3]:
#w = ["red", "yellow", "black", "brown", "red", "black", "black", "champion"]
#m = ["a","b","c","d","e","f","g","h"]
#t = {"Winners": w, "Maps":m, "Complete":True}
#util.get_matches(t)

In [4]:
# Get new data
#_ = util.get_data(100, unpickle=False)

In [25]:
def get_deviations(a):
    deviations = np.zeros(a.shape)
    for i in range(a.shape[0]):
        p = a[i]
        deviation = abs(p - 0.5)
        deviations[i] = deviation
    return deviations

In [5]:
x,y = util.get_vecs()
print(len(x))

9352


In [6]:
# Augment the vectors with the symmetrical data interpretation
def augment_vecs(x,y,n=None):
    n_maps = len(util.get_maps())
    unit_len = len(util.get_sorted_attributes()) + 2
    team_len = 8*unit_len
    for i,(match, label) in enumerate(zip(x,y)):
        if n is not None and i == n:
            break
        team1 = match[:team_len]
        team2 = match[team_len:2*team_len]
        m_map = match[2*team_len:]
        new_match = np.concatenate([team2, team1, m_map])
        new_label = abs(1-label)
        x.append(new_match)
        y.append(label)

In [7]:
#augment_vecs(x,y,200)

In [8]:
n_maps = len(util.get_maps())
unit_len = len(util.get_sorted_attributes()) + 2
assert unit_len * 8 + n_maps == 6654

## Keras

In [9]:
import keras
import sklearn
import sklearn.model_selection

Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarn

In [10]:
from keras.layers import Dense, Dropout, LeakyReLU, Concatenate
from keras.models import Sequential, Model
from keras import regularizers

In [11]:
#from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = sklearn.model_selection.train_test_split(x, y, test_size=0.2)
x_train = np.array(x_train)
x_test = np.array(x_test)
print(x_train.shape)

(7481, 6654)


In [32]:
from sklearn.neural_network import MLPRegressor
mlp = MLPRegressor(hidden_layer_sizes=(256,128,128,128), activation="relu", solver="adam", max_iter=500)
mlp.fit(x_train, y_train)

MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
             beta_2=0.999, early_stopping=False, epsilon=1e-08,
             hidden_layer_sizes=(256, 128, 128, 128), learning_rate='constant',
             learning_rate_init=0.001, max_fun=15000, max_iter=500,
             momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
             power_t=0.5, random_state=None, shuffle=True, solver='adam',
             tol=0.0001, validation_fraction=0.1, verbose=False,
             warm_start=False)

In [33]:
from sklearn.metrics import classification_report,confusion_matrix
predict_test = mlp.predict(x_test)
d = get_deviations(predict_test)
print(d.mean())
predict_round = np.round(predict_test)

#print(confusion_matrix(y_test,predict_round))
print(classification_report(y_test, predict_round))

0.29560810148331373
              precision    recall  f1-score   support

           0       0.53      0.58      0.55       894
           1       0.58      0.52      0.55       977

    accuracy                           0.55      1871
   macro avg       0.55      0.55      0.55      1871
weighted avg       0.55      0.55      0.55      1871



In [177]:
model = Sequential()
#model.add(Dense(1024, activation="relu", input_shape=(6654,), kernel_regularizer=regularizers.l2(0.001)))
model.add(Dense(1024, activation=LeakyReLU(alpha=0.1), input_shape=(6654,), kernel_regularizer=regularizers.l2(0.001)))
model.add(Dropout(0.5))
#model.add(Dense(512, activation="relu"))
model.add(Dense(512, activation=LeakyReLU(alpha=0.1)))
model.add(Dropout(0.5))
#model.add(Dense(256, activation="relu"))
model.add(Dense(256, activation=LeakyReLU(alpha=0.1)))
model.add(Dropout(0.5))
model.add(Dense(1, activation="sigmoid"))
model.summary()

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_45 (Dense)             (None, 1024)              6814720   
_________________________________________________________________
dropout_29 (Dropout)         (None, 1024)              0         
_________________________________________________________________
dense_46 (Dense)             (None, 512)               524800    
_________________________________________________________________
dropout_30 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_47 (Dense)             (None, 256)               131328    
_________________________________________________________________
dropout_31 (Dropout)         (None, 256)               0         
_________________________________________________________________
dense_48 (Dense)             (None, 1)               

/usr/local/lib/python3.6/dist-packages/keras/activations.py:235: UserWarning: Do not pass a layer instance (such as LeakyReLU) as the activation argument of another layer. Instead, advanced activation layers should be used just like any other layer in a model.
  identifier=identifier.__class__.__name__))


In [179]:
model = Sequential()
#model.add(Dense(1024, activation="relu", input_shape=(6654,), kernel_regularizer=regularizers.l2(0.001)))
model.add(Dense(16, activation="relu", input_shape=(6654,), kernel_regularizer=regularizers.l2(0.001)))
model.add(Dropout(0.2))
model.add(Dense(8, activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(1, activation="sigmoid"))
model.summary()

Model: "sequential_18"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_115 (Dense)            (None, 16)                106480    
_________________________________________________________________
dropout_12 (Dropout)         (None, 16)                0         
_________________________________________________________________
dense_116 (Dense)            (None, 8)                 136       
_________________________________________________________________
dropout_13 (Dropout)         (None, 8)                 0         
_________________________________________________________________
dense_117 (Dense)            (None, 1)                 9         
Total params: 106,625
Trainable params: 106,625
Non-trainable params: 0
_________________________________________________________________


In [10]:
def certainty_loss(layer, alpha, beta):
    def loss(y_true, y_pred):
        pred_err = keras.backend.mean(keras.backend.square(y_pred - y_true))
        cert_err = keras.backend.mean(keras.backend.square(layer[1]))
        cert_pen = keras.backend.mean(keras.backend.square((y_pred - y_true) * layer[1]))
        return pred_err + alpha * cert_err + beta * cert_pen
    return loss

In [21]:
def make_unit_model(unit_len):
    i = keras.layers.Input(shape=(unit_len,))
    #d = Dense(16, activation="relu", input_shape=(length,), kernel_regularizer=regularizers.l2(0.001))
    #d = Dense(unit_len, activation="relu", kernel_regularizer=regularizers.l2(0.005))(i)
    d = Dense(unit_len, activation="relu")(i)
    #d = Dropout(0.5)(d)
    #d = Dense(unit_len, activation="relu")(i)
    #d = Dropout(0.5)(d)
    #d = Dense(64, activation="relu", kernel_regularizer=regularizers.l2(0.005))(d)
    d = Dense(256, activation="relu")(d)
    #d = Dropout(0.5)(d)
    #d = Dense(64, activation="relu", kernel_regularizer=regularizers.l2(0.005))(d)
    d = Dense(128, activation="relu")(d)
    #d = Dropout(0.5)(d)
    #d = Dense(16, activation="relu"(d)
    m = Model(i,d)
    return m

def make_map_model(map_len):
    i = keras.layers.Input(shape=(n_maps,))
    d = Dense(n_maps, activation="relu")(i)
    d = Dense(n_maps, activation="relu")(i)
    m = Model(i,d)
    return m

unit_model = make_unit_model(unit_len)
map_model = make_map_model(n_maps)

# "Convolutional" in each unit
unit_inputs = [keras.layers.Input(shape=(unit_len,)) for _ in range(8)]
map_input = keras.layers.Input(shape=(n_maps,))
first_nodes = [unit_model(input_node) for input_node in unit_inputs]
map_nodes = map_model(map_input)
all_inputs = unit_inputs + [map_input]
#all_inputs = unit_inputs

In [73]:
#print(len(first_nodes))
c = keras.layers.Concatenate()(first_nodes + [map_nodes])
#c = keras.layers.Concatenate()(first_nodes)
#c = Dropout(0.5)(c)
#c = Dense(16, activation="relu", kernel_regularizer=regularizers.l2(0.01))(c)
#c = Dense(256, activation="relu", kernel_regularizer=regularizers.l2(0.01))(c)
c = Dense(512, activation="relu")(c)
c = Dropout(0.5)(c)
#c = Dense(128, activation="relu", kernel_regularizer=regularizers.l2(0.01))(c)
c = Dense(256, activation="relu")(c)
c = Dropout(0.5)(c)
#c = Dense(64, activation="relu", kernel_regularizer=regularizers.l2(0.01))(c)
c = Dense(128, activation="relu")(c)
c = Dropout(0.5)(c)
output = Dense(1, activation="sigmoid")(c) # Change to 2 for correct use of certainty loss
model = Model(inputs=all_inputs, outputs=output)
model.summary()

Model: "model_13"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_14 (InputLayer)           (None, 817)          0                                            
__________________________________________________________________________________________________
input_15 (InputLayer)           (None, 817)          0                                            
__________________________________________________________________________________________________
input_16 (InputLayer)           (None, 817)          0                                            
__________________________________________________________________________________________________
input_17 (InputLayer)           (None, 817)          0                                            
___________________________________________________________________________________________

In [51]:
def mk_multi(data):
    units = [data[:,(i * unit_len):(i+1) * unit_len] for i in range(8)]
    maps = data[:,8 * unit_len:]
    return units + [maps]

def get_multi_sample(multi_data, n):
    l = []
    for i in multi_data:
        a = i[n,:]
        a = np.expand_dims(a,0)
        l.append(a)
    return l

In [52]:
multi_train = mk_multi(x_train)
multi_test = mk_multi(x_test)

In [74]:
# Train the model
#model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
history = model.fit(x=multi_train, y=y_train, batch_size=100, epochs=10, verbose=True, validation_data=(multi_test, y_test))

Train on 7481 samples, validate on 1871 samples
Epoch 1/10
7481/7481 [==============================] - 3s 393us/step - loss: 0.2509 - accuracy: 0.9353 - val_loss: 2.3580 - val_accuracy: 0.5521
Epoch 2/10
7481/7481 [==============================] - 2s 313us/step - loss: 0.0488 - accuracy: 0.9884 - val_loss: 3.3744 - val_accuracy: 0.5521
Epoch 3/10
7481/7481 [==============================] - 2s 315us/step - loss: 0.0184 - accuracy: 0.9941 - val_loss: 4.3081 - val_accuracy: 0.5478
Epoch 4/10
7481/7481 [==============================] - 2s 311us/step - loss: 0.0125 - accuracy: 0.9969 - val_loss: 4.7062 - val_accuracy: 0.5500
Epoch 5/10
7481/7481 [==============================] - 2s 325us/step - loss: 0.0094 - accuracy: 0.9979 - val_loss: 4.8020 - val_accuracy: 0.5505
Epoch 6/10
7481/7481 [==============================] - 2s 318us/step - loss: 0.0202 - accuracy: 0.9960 - val_loss: 4.2411 - val_accuracy: 0.5612
Epoch 7/10
7481/7481 [==============================] - 2s 316us/step - loss

In [22]:
# Has dropout
NN = keras.backend.function(model.input + [keras.backend.learning_phase()], model.output)
model.input

[<tf.Tensor 'input_3:0' shape=(?, 817) dtype=float32>,
 <tf.Tensor 'input_4:0' shape=(?, 817) dtype=float32>,
 <tf.Tensor 'input_5:0' shape=(?, 817) dtype=float32>,
 <tf.Tensor 'input_6:0' shape=(?, 817) dtype=float32>,
 <tf.Tensor 'input_7:0' shape=(?, 817) dtype=float32>,
 <tf.Tensor 'input_8:0' shape=(?, 817) dtype=float32>,
 <tf.Tensor 'input_9:0' shape=(?, 817) dtype=float32>,
 <tf.Tensor 'input_10:0' shape=(?, 817) dtype=float32>,
 <tf.Tensor 'input_11:0' shape=(?, 118) dtype=float32>]

In [84]:
a = model.predict(multi_test)
d = get_deviations(a)
print(d.mean())
print(sum(y_test) / len(y_test))

0.5
0.5120256547300909


In [359]:
def predict_with_uncertainty(f, x, n_iterations):
    result = np.zeros((n_iterations,))
    for i in range(n_iterations):
        result[i] = f(x + [1])
    prediction = result.mean()
    uncertainty = result.var()
    return prediction, uncertainty

n_samples = 100
predictions = []
uncertainties = []
for i in range(multi_test[0].shape[0]):
    #print(i)
    labels = []
    x = get_multi_sample(multi_test, i)
    p,u = predict_with_uncertainty(NN,x,n_samples)
    predictions.append(p)
    uncertainties.append(u)
len(uncertainties)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

1824

In [361]:
# Did this all actually work?
l = len(uncertainties)
indexes = zip(range(l), uncertainties)
indexes_s = sorted(indexes, key=lambda x: x[1])

In [362]:
# The lowest variance ones appear at the front
indexes_s[:10]

# Does the model actually have higher accuracy for more certain indices?
def select_indices_array(a, indices):
    out = []
    for i,_ in indices:
        #print(i)
        #a[i,:]
        out.append(a[i,:])
    return np.array(out)

def select_indices_list(l, indices):
    out = []
    for i,_ in indices:
        out.append(l[i])
    return out

In [371]:
certain_indices = indexes_s[:200]
x_certain_test = select_indices_array(x_test, certain_indices)
y_certain_test = select_indices_list(y_test, certain_indices)
x_c_multi = mk_multi(x_certain_test)
model.evaluate(x_c_multi, y_certain_test)

200/200 [==============================] - 0s 235us/step


[2.549534592628479, 0.8149999976158142]

In [380]:
n = model.predict(x_c_multi)
#print(n[0])
l = []
for i in range(len(n)):
    # We got it wrong
    if round(n[i][0]) != y_certain_test[i]:
        #print(n[i])
        #print(y_certain_test[i])
        #print(i)
        #print(x_c_multi[i])
        x = get_multi_sample(multi_test, i)
        p,u = predict_with_uncertainty(NN, x, 100)
        l.append(p)
print(len(l))
#l = []
#for i in n:

[1.]
0
0
[0.]
1
15
[1.]
0
18
[1.]
0
23
[5.9604645e-08]
1
31
[0.99999976]
0
39
[0.99999917]
0
48
[0.9999989]
0
55
[1.1920929e-06]
1
58
[0.99999905]
0
63
[0.99999976]
0
78
[0.9999993]
0
80
[0.99999934]
0
83
[0.9999966]
0
87
[0.9999982]
0
88
[0.99999714]
0
93
[0.9999982]
0
94
[0.99999154]
0
100
[0.9999925]
0
105
[0.9999946]
0
111
[0.9999988]
0
117
[0.9999901]
0
121
[0.99999046]
0
122
[2.1338463e-05]
1
123
[0.9999966]
0
124
[0.9999975]
0
131
[0.9999997]
0
136
[0.99997365]
0
139
[0.99999607]
0
153
[0.9999862]
0
161
[7.0631504e-06]
1
167
[0.9999982]
0
169
[0.99999714]
0
179
[0.9999875]
0
184
[3.260374e-05]
1
189
[0.9999887]
0
191
[0.9999927]
0
198
37


In [386]:
l_n = np.array(l)
print(l_n.mean())
print(l_n.var())
print(l_n)

0.5305881735530615
0.20358492673689357
[5.35857612e-01 7.72481565e-01 9.85129995e-01 1.29775665e-02
 9.97602803e-01 7.32664516e-04 6.61180780e-02 3.06518553e-01
 9.86309744e-01 3.79115613e-03 9.97224919e-01 2.47649303e-02
 9.99510775e-01 9.40669724e-01 9.70880021e-01 9.93621162e-01
 9.99965966e-01 9.09207220e-01 9.99168744e-01 2.49230262e-02
 3.98033447e-02 1.73387414e-02 9.95315329e-01 9.95733646e-01
 9.99691629e-01 5.43785328e-03 9.92191378e-01 1.94349562e-02
 1.14546720e-01 1.83452872e-03 1.21534278e-03 4.64864628e-05
 9.99486297e-01 9.98992798e-01 1.04409373e-01 2.23311078e-01
 5.95516697e-01]


In [366]:
uncertain_indices = indexes_s[-200:]
x_uncertain_test = select_indices_array(x_test, uncertain_indices)
y_uncertain_test = select_indices_list(y_test, uncertain_indices)
x_uc_multi = mk_multi(x_uncertain_test)
model.evaluate(x_uc_multi, y_certain_test)

200/200 [==============================] - 0s 230us/step


[0.8245792865753174, 0.49000000953674316]

In [ ]:
# YAY? Higher accuracy on the elements with lower variance!

## Metric Learning

In [80]:
from metric_learn import NCA, LMNN, LFDA, MLKR
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline

In [57]:
#nca = NCA(max_iter=200)
#nca.fit(x,y)

In [81]:
clf = make_pipeline(NCA(max_iter=100), KNeighborsClassifier(weights="distance"))

In [85]:
c = cross_val_score(clf, x[:200], y[:200], verbose=True)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/usr/local/lib/python3.6/dist-packages/metric_learn/mlkr.py:181: ChangedBehaviorWarning: Warning, no init was set (`init=None`). As of version 0.5.0, the default init will now be set to 'auto', instead of 'pca'. If you still want to use PCA as an init, set init='pca'. This warning will disappear in v0.6.0, and `init` parameter's default value will be set to 'auto'.
  warnings.warn(msg, ChangedBehaviorWarning)
/usr/local/lib/python3.6/dist-packages/metric_learn/mlkr.py:181: ChangedBehaviorWarning: Warning, no init was set (`init=None`). As of version 0.5.0, the default init will now be set to 'auto', instead of 'pca'. If you still want to use PCA as an init, set init='pca'. This warning will disappear in v0.6.0, and `init` parameter's default value will be set to 'auto'.
  warnings.warn(msg, ChangedBehaviorWarning)
/usr/local/lib/python3.6/dist-packages/metric_learn/mlkr.py:181: ChangedBehaviorWarning: Warn

In [86]:
c, c.mean()

(array([0.65 , 0.525, 0.5  , 0.5  , 0.5  ]), 0.5349999999999999)

## SVM

In [ ]:
from sklearn import svm
from sklearn import metrics

In [159]:
svm_model = svm.SVC(kernel="rbf", verbose=True)
svm_model.fit(x_train, y_train)

[LibSVM]

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=True)

In [160]:
y_pred = svm_model.predict(x_test)
print("Accuracy: {}".format(metrics.accuracy_score(y_test, y_pred)))

Accuracy: 0.5355249204665959
